In [ ]:
"""
┌──────────────────────────────┐          ┌──────────────────────────────┐
│          Usuario             │          │    Respuesta generada en     │
│ Sube un PDF y hace preguntas │          │      lenguaje natural        │
└────────────┬─────────────────┘          └────────────▲─────────────────┘
             │                                         │
             ▼                                         │
┌──────────────────────────────┐          ┌──────────────────────────────┐
│          Streamlit           │          │  LangChain + Ollama (gemma)  │
│ Interfaz web (UI/UX)         │          │  LLM local via Ollama        │
└────────────┬────────────────┘           └────────────▲─────────────────┘
             │                                         │
             ▼                                         │
┌──────────────────────────────┐          ┌──────────────────────────────┐
│     Extracción de texto      │          │      Búsqueda y envío        │
│    desde PDF (pdfplumber)    │          │     del contexto al LLM      │
└────────────┬────────────────┘           └────────────▲─────────────────┘
             │                                         │
             ▼                                         │
┌──────────────────────────────┐          ┌──────────────────────────────┐
│  Generación de embeddings    │          │        Indexación semántica  │
│ con HuggingFaceEmbeddings    │ ────►    │          usando FAISS        │
└──────────────────────────────┘          └──────────────────────────────┘

"""

In [49]:
from langdetect import detect, DetectorFactory
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryMemory
from langchain.prompts import PromptTemplate
from PyPDF2 import PdfReader

from langchain_community.llms import Ollama
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

In [50]:
# Step 1: Load & extract text
pdf_path = "..\\data\\La ciudad de Nuvora.pdf"
reader = PdfReader(pdf_path)
raw_text = "\n".join(page.extract_text() for page in reader.pages if page.extract_text())

In [51]:
raw_text

' La ciudad de Nuvora: una utopía tecnológica en el año 2147  \nEn el año 2147, la ciudad flotante de Nuvora se convirtió en el primer asentamiento \nhumano construido completamente sobre una plataforma aérea sustentada por energía \ncuántica. Diseñada por el ingeniero aeroespacial Lin Mei -Han y el filósofo urbano Gabriel \nLurentz, Nuvora fue concebida como un experimento sociotecnológico donde inteligencia \nartificial, democracia directa y sostenibilidad conviven en equilibrio.  \nLa ciudad no tiene gobierno central. Todas las decisiones se toman mediante votación \nciudadana a través de un sistema llamado Círculo de Conciencia Colectiva  (CCC), el cual \ncombina algoritmos de consenso con datos recogidos por sensores distribuidos en toda la \nciudad. Este sistema predice consecuencias potenciales de cada decisión y presenta \nsimulaciones interactivas antes de que los ciudadanos voten.  \nLa economía de Nuvora es postmonetaria: en lugar de dinero, los ciudadanos \nintercambian "ti

In [52]:
try:
    detected_lang = detect(raw_text[:1000])
except:
    detected_lang = "unknown"
language = detected_lang
language

'es'

In [53]:
OLLAMA_MODEL = "gemma:2b"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100
NUM_TOP_DOCS = 3

# Load local LLM
def load_llm():
    return Ollama(model=OLLAMA_MODEL, temperature=0.1)

# Load PDF, split and embed
def process_pdf(path):
    loader = PyPDFLoader(path)
    pages = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    docs = splitter.split_documents(pages)

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(docs, embeddings)
    retriever = vectorstore.as_retriever()
    return retriever

In [54]:
# Load LLM and retriever
llm = load_llm()
retriever = process_pdf(pdf_path)

In [55]:
# Get summary
summary = llm.predict(f"Resuma el documento en 3 puntos:\n{raw_text[:3000]}")
summary

'1. La ciudad de Nuvora es un asentamiento humano construido sobre una plataforma aérea sustentada por energía cuántica.\n\n\n2. La ciudad no tiene gobierno central y las decisiones se toman mediante votación ciudadana a través del Círculo de Conciencia Colectiva.\n\n\n3. La economía de Nuvora es postmonetaria, en la que los ciudadanos intercambian "tiempo de dedicación" por "tiempo de dedicación".'

In [56]:
CUSTOM_PROMPT = """
Eres un asistente inteligente con acceso a un documento y conocimientos generales.
Intente siempre responder utilizando el documento, pero si éste no contiene la respuesta,
no dude en responder con conocimientos generales útiles.

Mantén un tono amistoso y conversacional, y que las respuestas sean breves y claras.
Por favor responde en Español

{context}

Chat History:
{chat_history}

User: {question}
"""

In [58]:
# Setup memory and prompt
chat_history = []
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)
prompt = PromptTemplate(
    input_variables=["chat_history", "question", "context"],
    template=CUSTOM_PROMPT
    )

In [59]:
# Build Conversational QA chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt},
    verbose=False
    )

In [61]:
user_input = "¿Qué usan los ciudadanos de Nuvora en lugar de dinero?"
response = qa_chain.invoke(user_input)
print(response)

{'question': '¿Qué usan los ciudadanos de Nuvora en lugar de dinero?', 'chat_history': [SystemMessage(content='The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.', additional_kwargs={}, response_metadata={})], 'answer': 'Los ciudadanos de Nuvora intercambian "tiempo de dedicación" para registrar y transfierir el esfuerzo socialmente útil que cada persona aporta a la comunidad.'}


In [ ]:
""" ¿En qué año fue construida la ciudad de Nuvora?
    ¿Quiénes diseñaron la ciudad de Nuvora?
    ¿Qué usan los ciudadanos de Nuvora en lugar de dinero?
    ¿Qué hacen con las personas que cometen delitos en Nuvora?
    ¿La cidudad tiene govierno central?
"""